In [ ]:
import Bio
import numpy as np
import sys
import os
import networkx as nx
import random as random
from tqdm import tqdm # loading bar
from utils_copy import linear_C, get_cost_2, get_sequence_string, parse_fasta_multiple, create_score_matrix, write_alignments_to_file, linear_backtrack, fill_graph,new_sp_approxi_combi
from utils_copy import convert_to_desired_format_nr_version, compute_cost, my_traversal_simply, extend_alignment_chaos, find_min_span_edges_testing, parse_fasta_multiple_remove_n
import timeit

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

filename='brca1-testseqs.fasta'
filename2='brca1-full.fasta'
brca_seqs_work_please,names=parse_fasta_multiple(filename)
brca_seqs_long=parse_fasta_multiple_remove_n(filename2)

In [2]:
def al_integrity_testt(A_dict, alignment_pairs, index_dict, MSA,verbose=False):
    integrity=True
    for key, value in alignment_pairs.items():
        position_key = index_dict[key] #find the position in the MSA
        position_value = index_dict[value]
        seq1 = []
        seq2 = []
        for element in MSA:
            seq1.append(element[int(position_key)]) #extract the 'column' for the
            seq2.append(element[int(position_value)])
        zipped = [list(e) for e in zip(seq1, seq2)] #zip the two lists to get the same format as in the pairwise alignment!
        zipped = [sublist for sublist in zipped if not all(element == '-' for element in sublist)]#removing all-gap columns

        if ord(key) < ord(value):
            key_for_A_dict = str(key) + "_" + str(value) #to reflect naming practice in the big function..
        else:
            key_for_A_dict = str(value) + "_" + str(key)

        # Add debugging print statement
        if verbose:
            print("key_for_A_dict:", key_for_A_dict)

        # Check if key_for_A_dict exists in A_dict
        if key_for_A_dict in A_dict:
            pair_al = A_dict[key_for_A_dict]
            for i in range(min(len(zipped), len(pair_al))): #iterate and compare columns!ch
                if zipped[i] != pair_al[i]:
                    print("A problem occurred: At position " + str(i) + ", " + str(zipped[i]) + " is not " + str(pair_al[i]))
                    integrity=False
            print("A problem occurred: key_for_A_dict not found in A_dict")
        if integrity and verbose:
            print("Alignment integrity looks fine!")
